In [ ]:
!pip install -q xgboost==1.7.6 lightgbm==3.3.5 shap==0.41.0 joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.9 MB/s eta 0:00:00


In [ ]:
!pip install numpy==1.26.4 shap==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have nu

# **SOIL_HEALTH**

In [13]:
# Modified combined-training script with anti-overfitting measures
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from joblib import dump, load
import warnings
warnings.filterwarnings("ignore")

SYNTHETIC_PATH = '/content/drive/MyDrive/FarmEasy/soil/soil_synthetic_rice_12000.csv'
SENSOR_PATH    = '/content/drive/MyDrive/FarmEasy/soil/sensor_data_with_fertility.csv'
OUTPUT_MODEL_DIR = '/content/drive/MyDrive/FarmEasy/soil/soil_model_combined_output_nooverfit'
os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)
RNG_SEED = 42

FEATURE_COLS = ['Moisture_%','Temperature_C','EC_uS_cm','pH','N_mg_per_kg','P_mg_per_kg','K_mg_per_kg']

sensor_rename_map = {
    "Moist (%)": "Moisture_%",
    "Temp (C)": "Temperature_C",
    "EC (uS/cm)": "EC_uS_cm",
    "N (mg/kg)": "N_mg_per_kg",
    "P (mg/kg)": "P_mg_per_kg",
    "K (mg/kg)": "K_mg_per_kg",
    "Timestamp": "Timestamp"
}

np.random.seed(RNG_SEED)

# ---------- 1) Load ----------
print("Loading datasets...")
df_csv = pd.read_csv(SYNTHETIC_PATH)
df_sensor = pd.read_csv(SENSOR_PATH)
df_sensor = df_sensor.rename(columns=sensor_rename_map)
print("Synthetic rows:", len(df_csv), "Sensor rows:", len(df_sensor))

# ---------- 2) Ensure feature columns exist ----------
for col in FEATURE_COLS:
    if col not in df_csv.columns:
        raise ValueError(f"Synthetic CSV missing required column {col}")
    if col not in df_sensor.columns:
        df_sensor[col] = np.nan  # we'll impute later

# ---------- 3) Targets ----------
def to_target_bin(series):
    if series.dtype.kind in 'iufc':
        return series.astype(int).replace({0:0,1:1})
    return series.map(lambda x: 1 if str(x).strip().lower() in ['fertile','1','true','yes','y'] else 0)

# synthetic expected to have 'target' = 'fertile'/'non_fertile'
if 'target' in df_csv.columns:
    df_csv['target_bin'] = to_target_bin(df_csv['target'])
else:
    raise ValueError("Synthetic CSV must have 'target' column")

# sensor: use provided target if exists, else create pseudo-labels (warn)
pseudo_label_used = False
if 'target' in df_sensor.columns:
    df_sensor['target_bin'] = to_target_bin(df_sensor['target'])
    print("Sensor contains 'target' column; using it.")
else:
    pseudo_label_used = True
    print("Sensor missing 'target' -> creating pseudo-labels using threshold rules (best-effort).")
    df_sensor['target_bin'] = (
        ((df_sensor['pH'] >= 5.5) & (df_sensor['pH'] <= 7.5)) &
        (df_sensor['EC_uS_cm'] < 1000) &
        (df_sensor['Temperature_C'] >= 20) & (df_sensor['Temperature_C'] <= 30) &
        (df_sensor['Moisture_%'] >= 30) &
        (((df_sensor['N_mg_per_kg'] < 20).fillna(True)).astype(int) + (df_sensor['P_mg_per_kg'] < 12).fillna(True).astype(int) + (df_sensor['K_mg_per_kg'] < 100).fillna(True).astype(int) <= 1)
    ).astype(int)

df_csv['source'] = 'csv'
df_sensor['source'] = 'sensor'

# ---------- 4) Concatenate (keep Timestamp if present in sensor) ----------
keep_cols = FEATURE_COLS + ['target_bin','source']
if 'Timestamp' in df_sensor.columns:
    df_sensor_out = df_sensor[FEATURE_COLS + ['target_bin','source','Timestamp']].copy()
else:
    df_sensor_out = df_sensor[keep_cols].copy()
df_csv_sel = df_csv[FEATURE_COLS + ['target_bin','source']].copy()
combined = pd.concat([df_csv_sel, df_sensor_out[keep_cols]], ignore_index=True, sort=False).reset_index(drop=True)

print("Combined size:", len(combined))
print(combined['source'].value_counts())

# ---------- 5) Compute sample weights (equal total weight per source) ----------
n_csv = (combined['source']=='csv').sum()
n_sensor = (combined['source']=='sensor').sum()
total = n_csv + n_sensor
w_csv = (0.5 * total) / max(n_csv,1)
w_sensor = (0.5 * total) / max(n_sensor,1)
combined['sample_weight'] = combined['source'].map({'csv': w_csv, 'sensor': w_sensor})
print(f"Per-row weights csv={w_csv:.4f}, sensor={w_sensor:.4f}")

# ---------- 6) Prepare features/labels and simple median imputation ----------
X_all = combined[FEATURE_COLS].copy()
y_all = combined['target_bin'].copy()
sw_all = combined['sample_weight'].values
X_all = X_all.fillna(X_all.median())

# ---------- 7) Train/test split ----------
X_train, X_test, y_train, y_test, sw_train, sw_test = train_test_split(
    X_all, y_all, sw_all, test_size=0.20, stratify=y_all, random_state=RNG_SEED
)
print("Train/Test sizes:", X_train.shape, X_test.shape)

# ---------- 8) Scale ----------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
dump(scaler, os.path.join(OUTPUT_MODEL_DIR, 'scaler.joblib'))

rf = RandomForestClassifier(
    n_estimators=250,
    max_depth=8,
    min_samples_leaf=20,
    max_features='sqrt',
    random_state=RNG_SEED,
    n_jobs=-1
)

xgb_clf = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    colsample_bytree=0.7,
    reg_lambda=2.0,
    random_state=RNG_SEED,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=4
)


n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RNG_SEED)


feature_std = X_train_scaled.std(axis=0)

AUG_NOISE_FACTOR = 0.03
rng = np.random.RandomState(RNG_SEED)

def get_oof_predictions_with_augment(clf, X_arr, y_series, X_test_arr, sample_weight_train=None):
    oof_train = np.zeros(X_arr.shape[0])
    test_preds_folds = np.zeros((X_test_arr.shape[0], n_splits))
    for i, (train_idx, val_idx) in enumerate(skf.split(X_arr, y_series)):
        X_tr, X_val = X_arr[train_idx].copy(), X_arr[val_idx].copy()
        y_tr = y_series.iloc[train_idx]

        noise_scale = feature_std * AUG_NOISE_FACTOR
        noise = rng.normal(0, noise_scale, size=X_tr.shape)
        X_tr_aug = X_tr + noise

        if sample_weight_train is not None:
            sw_tr = sample_weight_train[train_idx]
            clf.fit(X_tr_aug, y_tr, sample_weight=sw_tr)
        else:
            clf.fit(X_tr_aug, y_tr)

        oof_train[val_idx] = clf.predict_proba(X_val)[:,1]
        test_preds_folds[:, i] = clf.predict_proba(X_test_arr)[:,1]
    test_preds = test_preds_folds.mean(axis=1)
    return oof_train.reshape(-1,1), test_preds.reshape(-1,1)

X_train_arr = X_train_scaled.copy()
X_test_arr = X_test_scaled.copy()
sw_train_arr = np.array(sw_train)

print("Generating OOF predictions for RandomForest (with small augment)...")
rf_oof_train, rf_test_pred = get_oof_predictions_with_augment(rf, X_train_arr, y_train, X_test_arr, sample_weight_train=sw_train_arr)

print("Generating OOF predictions for XGBoost (with small augment)...")
xgb_oof_train, xgb_test_pred = get_oof_predictions_with_augment(xgb_clf, X_train_arr, y_train, X_test_arr, sample_weight_train=sw_train_arr)

X_meta_train = np.hstack([rf_oof_train, xgb_oof_train])
X_meta_test  = np.hstack([rf_test_pred, xgb_test_pred])

meta = LogisticRegression(C=0.1, max_iter=2000)
meta.fit(X_meta_train, y_train, sample_weight=sw_train_arr)

calibrator = CalibratedClassifierCV(estimator=meta, method='sigmoid', cv=5)
calibrator.fit(X_meta_train, y_train)

train_proba = calibrator.predict_proba(X_meta_train)[:,1]
train_pred = (train_proba >= 0.5).astype(int)

print("\n--- TRAIN (OOF) METRICS ---")
print(classification_report(y_train, train_pred, target_names=['non_fertile','fertile']))
print("ROC AUC (train):", round(roc_auc_score(y_train, train_proba),4))
print("Brier (train):", round(brier_score_loss(y_train, train_proba),6))
print("Confusion (train):\n", confusion_matrix(y_train, train_pred))

rf.fit(X_train_arr, y_train, sample_weight=sw_train_arr)

X_xgb_tr, X_xgb_val, y_xgb_tr, y_xgb_val, sw_xgb_tr, sw_xgb_val = train_test_split(
    X_train_arr, y_train, sw_train_arr, test_size=0.15, stratify=y_train, random_state=RNG_SEED)

dtrain = xgb.DMatrix(X_xgb_tr, label=y_xgb_tr, weight=sw_xgb_tr)
dval = xgb.DMatrix(X_xgb_val, label=y_xgb_val, weight=sw_xgb_val)

params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'lambda': 2.0,
    'eval_metric': 'logloss',
    'seed': RNG_SEED,
    'nthread': 4,
    'disable_default_get_object': 1
}

xgb_booster_final = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dval, 'validation')],
    early_stopping_rounds=30,
    verbose_eval=False
)


class XGBBoosterWrapper:
    def __init__(self, booster):
        self.booster = booster
        self.best_iteration = getattr(booster, "best_iteration", None)

    def predict_proba(self, X):
        d = xgb.DMatrix(X)
        p = self.booster.predict(d)
        return np.vstack([1 - p, p]).T

xgb_wrapper = XGBBoosterWrapper(xgb_booster_final)

rf_test_proba  = rf.predict_proba(X_test_arr)[:,1]
xgb_test_proba = xgb_wrapper.predict_proba(X_test_arr)[:,1]
X_meta_test_final = np.column_stack([rf_test_proba, xgb_test_proba])
calib_test_proba = calibrator.predict_proba(X_meta_test_final)[:,1]
test_pred = (calib_test_proba >= 0.5).astype(int)

print("\n--- TEST METRICS ---")
print(classification_report(y_test, test_pred, target_names=['non_fertile','fertile']))
print("ROC AUC (test):", round(roc_auc_score(y_test, calib_test_proba),4))
print("Brier (test):", round(brier_score_loss(y_test, calib_test_proba),6))
print("Confusion (test):\n", confusion_matrix(y_test, test_pred))

dump(scaler, os.path.join(OUTPUT_MODEL_DIR,'scaler.joblib'))
dump(rf, os.path.join(OUTPUT_MODEL_DIR,'rf_model.joblib'))

xgb_booster_final.save_model(os.path.join(OUTPUT_MODEL_DIR,'xgb_model.json'))

dump(meta, os.path.join(OUTPUT_MODEL_DIR,'meta_lr.joblib'))
dump(calibrator, os.path.join(OUTPUT_MODEL_DIR,'calibrator.joblib'))
print("\nSaved models to:", OUTPUT_MODEL_DIR)

print("\nTrain label distribution:", y_train.value_counts().to_dict())
print("Test label distribution:", y_test.value_counts().to_dict())
if pseudo_label_used:
    print("\nNote: sensor data lacked 'target' labels — pseudo-labels were used for sensor rows.")

def nutrient_suggestions(row):
    issues = []
    if row['pH'] < 5.5 or row['pH'] > 7.5:
        issues.append(f"pH_out_of_range ({row['pH']})")
    if row['EC_uS_cm'] >= 1000:
        issues.append(f"High_EC ({row['EC_uS_cm']})")
    if row['Temperature_C'] < 20.0 or row['Temperature_C'] > 30.0:
        issues.append(f"Temperature_off ({row['Temperature_C']} °C)")
    if row['Moisture_%'] < 30.0:
        issues.append(f"Low_moisture ({row['Moisture_%']}%)")
    if row['N_mg_per_kg'] < 20.0:
        issues.append("N_def")
    if row['P_mg_per_kg'] < 12.0:
        issues.append("P_def")
    if row['K_mg_per_kg'] < 100.0:
        issues.append("K_def")
    return issues

loaded_scaler = load(os.path.join(OUTPUT_MODEL_DIR, 'scaler.joblib'))
loaded_rf = load(os.path.join(OUTPUT_MODEL_DIR, 'rf_model.joblib'))

loaded_xgb_booster = xgb.Booster()
loaded_xgb_booster.load_model(os.path.join(OUTPUT_MODEL_DIR,'xgb_model.json'))
loaded_calibrator = load(os.path.join(OUTPUT_MODEL_DIR, 'calibrator.joblib'))

def predict_sensor_reading(reading_dict):

    row_df = pd.DataFrame([reading_dict])

    row_df_filled = row_df[FEATURE_COLS].fillna(X_all.median().loc[FEATURE_COLS])

    Xr_scaled = loaded_scaler.transform(row_df_filled.values)

    p_rf = loaded_rf.predict_proba(Xr_scaled)[:,1]

    dX = xgb.DMatrix(Xr_scaled)
    p_xgb = loaded_xgb_booster.predict(dX)

    p_calib = loaded_calibrator.predict_proba(np.column_stack([p_rf, p_xgb]))[:,1]
    label = 'fertile' if p_calib[0] >= 0.5 else 'non_fertile'
    issues = nutrient_suggestions(reading_dict)
    return {'prob_fertile': float(p_calib[0]), 'label': label, 'issues': issues}

example = combined.loc[combined['source']=='sensor'].iloc[0][FEATURE_COLS].to_dict()
print("\nExample sensor prediction:", predict_sensor_reading(example))

Loading datasets...
Synthetic rows: 12000 Sensor rows: 263
Sensor contains 'target' column; using it.
Combined size: 12263
source
csv       12000
sensor      263
Name: count, dtype: int64
Per-row weights csv=0.5110, sensor=23.3137
Train/Test sizes: (9810, 7) (2453, 7)
Generating OOF predictions for RandomForest (with small augment)...
Generating OOF predictions for XGBoost (with small augment)...

--- TRAIN (OOF) METRICS ---
              precision    recall  f1-score   support

 non_fertile       0.99      0.99      0.99      6796
     fertile       0.99      0.99      0.99      3014

    accuracy                           0.99      9810
   macro avg       0.99      0.99      0.99      9810
weighted avg       0.99      0.99      0.99      9810

ROC AUC (train): 0.9996
Brier (train): 0.006122
Confusion (train):
 [[6752   44]
 [  38 2976]]

--- TEST METRICS ---
              precision    recall  f1-score   support

 non_fertile       1.00      1.00      1.00      1699
     fertile      

# **TESTING**

In [14]:

import numpy as np
import xgboost as xgb
from joblib import load
import pandas as pd

# ----- CONFIG -----
MODEL_DIR = "/content/drive/MyDrive/FarmEasy/soil/soil_model_combined_output_nooverfit"

FEATURE_COLS = [
    'Moisture_%','Temperature_C','EC_uS_cm','pH',
    'N_mg_per_kg','P_mg_per_kg','K_mg_per_kg'
]

# ---------- Load saved models ----------
print("Loading models...")

scaler = load(f"{MODEL_DIR}/scaler.joblib")
rf_model = load(f"{MODEL_DIR}/rf_model.joblib")

xgb_booster = xgb.Booster()
xgb_booster.load_model(f"{MODEL_DIR}/xgb_model.json")

calibrator = load(f"{MODEL_DIR}/calibrator.joblib")

print("Models loaded successfully.\n")

# ---------- Nutrient rule-based assistance ----------
def nutrient_suggestions(row):
    issues = []
    if row['pH'] < 5.5 or row['pH'] > 7.5:
        issues.append("⚠ pH out of ideal range (5.5–7.5)")
    if row['EC_uS_cm'] >= 1000:
        issues.append("⚠ High EC (salinity too high)")
    if row['Temperature_C'] < 20 or row['Temperature_C'] > 30:
        issues.append("⚠ Temperature not ideal (20–30°C)")
    if row['Moisture_%'] < 30:
        issues.append("⚠ Soil moisture too low (<30%)")
    if row['N_mg_per_kg'] < 20:
        issues.append("⚠ Nitrogen deficiency")
    if row['P_mg_per_kg'] < 12:
        issues.append("⚠ Phosphorus deficiency")
    if row['K_mg_per_kg'] < 100:
        issues.append("⚠ Potassium deficiency")
    return issues

# ---------- Function to predict from user input ----------
def predict_one():
    print("\nEnter soil readings:\n")

    user_input = {}
    for feat in FEATURE_COLS:
        user_input[feat] = float(input(f"Enter {feat}: "))

    # Convert to DataFrame
    df = pd.DataFrame([user_input])

    # Scale (model expects scaled)
    X_scaled = scaler.transform(df.values)

    # RandomForest probability
    p_rf = rf_model.predict_proba(X_scaled)[:,1]

    # XGBoost probability
    dmat = xgb.DMatrix(X_scaled)
    p_xgb = xgb_booster.predict(dmat)

    # Stacked probability (RF + XGB → Calibrator)
    p_final = calibrator.predict_proba(np.column_stack([p_rf, p_xgb]))[:,1][0]

    label = "fertile" if p_final >= 0.5 else "non_fertile"

    issues = nutrient_suggestions(user_input)

    print("\n---------------------------")
    print(" **SOIL FERTILITY RESULT**")
    print("---------------------------")
    print(f" Probability fertile: {p_final:.4f}")
    print(f" Final label: {label.upper()}")

    if issues:
        print("\n⚠ Nutrient / Condition Issues:")
        for i in issues:
            print(" -", i)
    else:
        print("\n No major issues detected.")

    print("---------------------------\n")

predict_one()


Loading models...
Models loaded successfully.


Enter soil readings:

Enter Moisture_%: 67
Enter Temperature_C: 16
Enter EC_uS_cm: 69
Enter pH: 5
Enter N_mg_per_kg: 45
Enter P_mg_per_kg: 32
Enter K_mg_per_kg: 12

---------------------------
🌾 **SOIL FERTILITY RESULT**
---------------------------
📌 Probability fertile: 0.0010
📌 Final label: NON_FERTILE

⚠ Nutrient / Condition Issues:
 - ⚠ pH out of ideal range (5.5–7.5)
 - ⚠ Temperature not ideal (20–30°C)
 - ⚠ Potassium deficiency
---------------------------

